In [4]:
import glob, os
from itertools import product
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn import preprocessing

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.neighbors import LocalOutlierFactor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans

import collections 
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold, PredefinedSplit
from sklearn.metrics import accuracy_score
import warnings 


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from pytimekr import pytimekr
warnings.filterwarnings(action="ignore")
%matplotlib inline

# Load the dataset
data = pd.read_csv('./train.csv')
data.head()

plt.rc("font", family="AppleGothic")
plt.rcParams['axes.unicode_minus'] = False


In [5]:
data_test = pd.read_csv('./test.csv')

In [6]:
data['item'].value_counts()

item
TG    15230
BC    13707
RD    12184
CR    10661
CB     7615
Name: count, dtype: int64

In [7]:
data.isna().sum()

ID             0
timestamp      0
item           0
corporation    0
location       0
supply(kg)     0
price(원/kg)    0
dtype: int64

In [8]:
type(data['timestamp'][0])

str

In [9]:
import datetime

data['timestamp'] = pd.to_datetime(data['timestamp'])

# 날짜 --> week와 day column 생성 (day는 요일을 나타냄)

data['year'] = data['timestamp'].dt.isocalendar().year
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['week'] = data['timestamp'].dt.isocalendar().week

data['weekday'] = data['timestamp'].dt.isocalendar().day



In [10]:
import datetime

data_test['timestamp'] = pd.to_datetime(data_test['timestamp'])

# 날짜 --> week와 day column 생성 (day는 요일을 나타냄)

data_test['year'] = data_test['timestamp'].dt.isocalendar().year
data_test['month'] = data_test['timestamp'].dt.month
data_test['day'] = data_test['timestamp'].dt.day
data_test['week'] = data_test['timestamp'].dt.isocalendar().week

data_test['weekday'] = data_test['timestamp'].dt.isocalendar().day



In [11]:
data['isWeekday'] = ((data['weekday'] >= 1) & (data['weekday'] <= 5)).astype(int)
data['isSaturday'] = (data['weekday'] == 6).astype(int)
data['isSunday'] = (data['weekday'] == 7).astype(int)

In [12]:
data_test['isWeekday'] = ((data_test['weekday'] >= 1) & (data_test['weekday'] <= 5)).astype(int)
data_test['isSaturday'] = (data_test['weekday'] == 6).astype(int)
data_test['isSunday'] = (data_test['weekday'] == 7).astype(int)

In [13]:
# weekday 삭제 
data.drop('weekday', axis=1,inplace=True)


In [14]:
data_test.drop('weekday', axis=1,inplace=True)

In [15]:
data

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,week,isWeekday,isSaturday,isSunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,1,1,0,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,1,1,0,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,1,1,0,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,9,1,0,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,9,1,0,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,9,1,0,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,9,1,0,0


In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [17]:
# Encoding categorical variables and scaling
categorical_cols = ['item', 'corporation', 'location', 'year','isWeekday','isSaturday','isSunday']
numeric_features = ['month', 'day', 'week']


encoder = OneHotEncoder(sparse=False)
encoded_train = encoder.fit_transform(data[categorical_cols])
encoded_test = encoder.transform(data_test[categorical_cols])

# Correcting the column selections by adding a comma between 'day' and 'week'
train_features = np.hstack((encoded_train, data[['month', 'day', 'week']].values))
test_features = np.hstack((encoded_test, data_test[['month', 'day', 'week']].values))


In [18]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_features)

In [20]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,14 hours 54 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_seungwoo_pu81ul
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.980 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [23]:
data.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,week,isWeekday,isSaturday,isSunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,1,1,0,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,1,1,0,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,1,1,0,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,1,0,1,0


In [26]:
train = pd.get_dummies(data, columns=['item', 'corporation', 'location', 'year'])
test = pd.get_dummies(data_test, columns=['item', 'corporation', 'location', 'year'])

In [27]:
train.head()

,ID,timestamp,supply(kg),price(원/kg),month,day,week,isWeekday,isSaturday,isSunday,...,corporation_D,corporation_E,corporation_F,location_J,location_S,year_2019,year_2020,year_2021,year_2022,year_2023
0,TG_A_J_20190101,2019-01-01,0.0,0.0,1,1,1,1,0,0,...,False,False,False,True,False,True,False,False,False,False
1,TG_A_J_20190102,2019-01-02,0.0,0.0,1,2,1,1,0,0,...,False,False,False,True,False,True,False,False,False,False
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,1,3,1,1,0,0,...,False,False,False,True,False,True,False,False,False,False
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,1,4,1,1,0,0,...,False,False,False,True,False,True,False,False,False,False
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,1,5,1,0,1,0,...,False,False,False,True,False,True,False,False,False,False


In [28]:
# Dropping the 'supply' column from the train_data
train_drop_supply = train.drop(columns=['supply(kg)'])

# Display the first few rows of the modified DataFrame
train_drop_supply.head()

,ID,timestamp,price(원/kg),month,day,week,isWeekday,isSaturday,isSunday,item_BC,...,corporation_D,corporation_E,corporation_F,location_J,location_S,year_2019,year_2020,year_2021,year_2022,year_2023
0,TG_A_J_20190101,2019-01-01,0.0,1,1,1,1,0,0,False,...,False,False,False,True,False,True,False,False,False,False
1,TG_A_J_20190102,2019-01-02,0.0,1,2,1,1,0,0,False,...,False,False,False,True,False,True,False,False,False,False
2,TG_A_J_20190103,2019-01-03,1728.0,1,3,1,1,0,0,False,...,False,False,False,True,False,True,False,False,False,False
3,TG_A_J_20190104,2019-01-04,1408.0,1,4,1,1,0,0,False,...,False,False,False,True,False,True,False,False,False,False
4,TG_A_J_20190105,2019-01-05,1250.0,1,5,1,0,1,0,False,...,False,False,False,True,False,True,False,False,False,False


In [29]:
train = train_drop_supply
train.columns.tolist()

['ID',
 'timestamp',
 'price(원/kg)',
 'month',
 'day',
 'week',
 'isWeekday',
 'isSaturday',
 'isSunday',
 'item_BC',
 'item_CB',
 'item_CR',
 'item_RD',
 'item_TG',
 'corporation_A',
 'corporation_B',
 'corporation_C',
 'corporation_D',
 'corporation_E',
 'corporation_F',
 'location_J',
 'location_S',
 'year_2019',
 'year_2020',
 'year_2021',
 'year_2022',
 'year_2023']

In [30]:
x = train.columns.tolist()
y = 'price(원/kg)'
x.remove(y)

In [31]:
# Import H2O and start the H2O server
import h2o
from h2o.automl import H2OAutoML
h2o.init()

# Assuming 'train' is your pandas DataFrame
h2o_train = h2o.H2OFrame(train)  # Convert the pandas DataFrame to an H2OFrame

# Now you can use h2o_train in your AutoML training
aml = H2OAutoML(max_models=10, seed=1, max_runtime_secs=1000)
aml.train(x=x, y=y, training_frame=h2o_train)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,14 hours 59 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_seungwoo_pu81ul
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.980 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
15:03:25.444: _train param, Dropping bad and constant columns: [ID]

████
15:03:59.452: _train param, Dropping bad and constant columns: [ID]


15:04:06.233: _train param, Dropping bad and constant columns: [ID]

█
15:04:31.318: _train param, Dropping bad and constant columns: [ID]

██
15:04:54.897: _train param, Dropping bad and constant columns: [ID]

███
15:05:40.832: _train param, Dropping bad and constant columns: [ID]

█
15:06:01.461: _train param, Dropping bad and constant columns: [ID]

█
15:06:16.278: _train param, Dropping bad and constant columns: [ID]

█
15:06:31.943: _train param, Dropping bad and constant columns: [ID]

███
15:07:30.138: _train param, Dropping bad and constant columns: [ID]

█
15:07:47.897: _train param, Dropping unused columns: [ID]

█
15:07:52.940: _train param, Dropping unused columns: [ID]

█████████████████████████████████████████████| (d

key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/10
# GBM base models (used / total),3/4
# XGBoost base models (used / total),2/3
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [32]:
lb = aml.leaderboard
print(lb)

model_id                                                    rmse     mse      mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231112_150325     815.519  665071  313.865  nan                          665071
StackedEnsemble_BestOfFamily_1_AutoML_1_20231112_150325  816.298  666342  312.401  nan                          666342
GBM_4_AutoML_1_20231112_150325                           820.502  673223  328.403  nan                          673223
GBM_3_AutoML_1_20231112_150325                           842.485  709781  359.092  nan                          709781
GBM_1_AutoML_1_20231112_150325                           846.041  715786  366.211  nan                          715786
DRF_1_AutoML_1_20231112_150325                           854.827  730730  288.467    2.13011                    730730
XGBoost_2_AutoML_1_20231112_150325                       855.247  731448  347.021  nan                          731448
GBM_2_AutoML_1_20231112_150325                  

In [33]:
test_h2o = h2o.H2OFrame(test)

preds = aml.leader.predict(test_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [34]:
predictions_df = h2o.as_list(preds)

In [35]:
h2o.shutdown()

H2O session _sid_9c5c closed.


/var/folders/xs/tvgnm55s5xjd3wv7fc3m0bv80000gn/T/ipykernel_15550/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


In [36]:
predictions_df

,predict
0,3258.109424
1,96.395758
2,3545.578570
3,3526.584666
4,3555.167064
...,...
1087,502.235555
1088,493.635403
1089,496.921017
1090,519.305003


In [37]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = predictions_df['predict']
submission.loc[data_test['isSunday'] == 1, 'answer'] = 0
submission['answer'][submission['answer'] < 0] = 0
# Save the updated predictions to a new CSV file
submission.to_csv('automl_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3258.109424
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3545.578570
3,TG_A_J_20230307,3526.584666
4,TG_A_J_20230308,3555.167064
...,...,...
1087,RD_F_J_20230327,502.235555
1088,RD_F_J_20230328,493.635403
1089,RD_F_J_20230329,496.921017
1090,RD_F_J_20230330,519.305003
